In [ ]:
import pandas as pd
import numpy as np

# =========================================================
# 1️⃣ Load data
# =========================================================
df = pd.read_csv("Rental_zip_raw_data.csv")

print("Original shape:", df.shape)
df.head(3)

# =========================================================
# 2️⃣ Basic cleanup
# =========================================================
df = df.dropna(how="all").drop_duplicates()

# Rename ZIP column for clarity
df.rename(columns={"RegionName": "ZipCode"}, inplace=True)

# Ensure ZIP is 5 digits (string)
df["ZipCode"] = df["ZipCode"].astype(str).str.zfill(5)

# Drop RegionType if it exists (keep only ZIP-level)
if "RegionType" in df.columns:
    df = df[df["RegionType"].str.lower() == "zip"].drop(columns=["RegionType"], errors="ignore")

# =========================================================
# 3️⃣ Identify date columns (those that look like YYYY-MM-DD)
# =========================================================
date_cols = [c for c in df.columns if c[:4].isdigit() and "-" in c]
id_cols = [c for c in df.columns if c not in date_cols]

print(f"Identified {len(date_cols)} date columns (e.g. {date_cols[:3]} ... {date_cols[-3:]})")

# =========================================================
# 4️⃣ Melt to long format
# =========================================================
long_df = df.melt(
    id_vars=id_cols,
    value_vars=date_cols,
    var_name="Date",
    value_name="Rent"
)

# Convert Date to datetime
long_df["Date"] = pd.to_datetime(long_df["Date"], errors="coerce")

# Drop invalid values
long_df = long_df.dropna(subset=["Rent"])
long_df = long_df[(long_df["Rent"] > 100) & (long_df["Rent"] < 10000)]

# Sort and reset index
long_df = long_df.sort_values(["ZipCode", "Date"]).reset_index(drop=True)

print("Cleaned + reshaped shape:", long_df.shape)
long_df.head(10)

# =========================================================
# 5️⃣ Save cleaned file
# =========================================================
output_path = "Rental_zip_cleaned_long.csv"
long_df.to_csv(output_path, index=False)
print(f"✅ Saved cleaned data to: {output_path}")


In [ ]:
!pip install prophet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet


# Load cleaned data
df = pd.read_csv("Rental_zip_cleaned_long.csv", parse_dates=["Date"])

# Make sure the expected columns exist
assert {"ZipCode", "Date", "Rent"}.issubset(df.columns)

print(df.shape)
df.head()


In [ ]:
# =========================================================
# 📦 1️⃣  IMPORT LIBRARIES
# =========================================================
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# =========================================================
# 🧾 2️⃣  LOAD CLEANED DATA
# =========================================================
df = pd.read_csv("Rental_zip_cleaned_long.csv", parse_dates=["Date"])
df.sort_values(["Metro", "ZipCode", "Date"], inplace=True)
assert {"ZipCode", "Date", "Rent", "Metro"}.issubset(df.columns)

print(f"✅ Loaded data for {df['ZipCode'].nunique()} ZIPs across {df['Metro'].nunique()} metros")

# =========================================================
# 🧮 3️⃣  IMPUTE SPARSE ZIPs USING METRO-LEVEL AVERAGES
# =========================================================
all_dates = pd.date_range(df["Date"].min(), df["Date"].max(), freq="ME")
filled = []

for metro, group in df.groupby("Metro"):
    pivot = group.pivot(index="Date", columns="ZipCode", values="Rent")
    pivot = pivot.reindex(all_dates)

    # Fill missing ZIPs each month using metro mean
    pivot = pivot.apply(lambda x: x.fillna(pivot.mean(axis=1)), axis=0)

    # Backfill + forward fill any early/late gaps
    pivot = pivot.bfill().ffill()

    # Back to long format
    melted = pivot.stack().reset_index()
    melted.columns = ["Date", "ZipCode", "Rent"]
    melted["Metro"] = metro
    filled.append(melted)

df_filled = pd.concat(filled, ignore_index=True)

print(f"✅ Imputed data shape: {df_filled.shape}")
print(f"🗓️ Date range: {df_filled['Date'].min().date()} → {df_filled['Date'].max().date()}")

# =========================================================
# 📈 4️⃣  FORECAST EACH ZIP USING TREND EXTRAPOLATION
# =========================================================
def forecast_zip_trend(zip_code, horizon=[3, 6, 9, 12]):
    """Forecast next N months using a linear trend."""
    zdf = df_filled[df_filled["ZipCode"] == zip_code].sort_values("Date")
    zdf["t"] = np.arange(len(zdf))
    X = zdf[["t"]].values  # ✅ Convert to numpy array to suppress warning
    y = zdf["Rent"].values

    model = LinearRegression()
    model.fit(X, y)

    future_t = np.arange(len(zdf), len(zdf) + max(horizon)).reshape(-1, 1)
    future_y = model.predict(future_t)  # ✅ feature names no longer relevant

    fc = pd.DataFrame({
        "ZipCode": zip_code,
        "Forecast_Month": horizon,
        "Forecast_Rent": future_y[np.array(horizon) - 1]
    })
    return fc

# =========================================================
# 🚀 5️⃣  RUN FORECASTS FOR ALL ZIPs
# =========================================================
all_forecasts = pd.concat(
    [forecast_zip_trend(z) for z in df_filled["ZipCode"].unique()],
    ignore_index=True
)

# Save output
all_forecasts.to_csv("Rental_forecasts_metro_based.csv", index=False)
print(f"✅ Forecasts generated for {all_forecasts['ZipCode'].nunique()} ZIPs")
print("📁 Saved: Rental_forecasts_metro_based.csv")

# =========================================================
# 📊 6️⃣  OPTIONAL: VISUAL CHECK FOR ONE ZIP
# =========================================================
example_zip = df_filled["ZipCode"].iloc[0]
hist = df_filled[df_filled["ZipCode"] == example_zip]
future_vals = all_forecasts[all_forecasts["ZipCode"] == example_zip]

plt.figure(figsize=(10, 4))
plt.plot(hist["Date"], hist["Rent"], label="Imputed + Historical")

# Convert Forecast_Month into approximate monthly offsets safely
future_dates = [
    hist["Date"].max() + pd.DateOffset(months=int(m))
    for m in future_vals["Forecast_Month"]
]

plt.scatter(
    future_dates,
    future_vals["Forecast_Rent"],
    color="orange",
    label="Forecast (3/6/9/12 mo)"
)

plt.title(f"ZIP {example_zip} — Metro-Imputed Trend Forecast")
plt.xlabel("Date")
plt.ylabel("Rent ($)")
plt.legend()
plt.show()



In [ ]:
# =========================================================
# 📦 1️⃣ IMPORT LIBRARIES
# =========================================================
import pandas as pd
import geopandas as gpd
import plotly.express as px
import os

# =========================================================
# 🧾 2️⃣ LOAD FORECAST DATA
# =========================================================
forecast_df = pd.read_csv("Rental_forecasts_metro_based.csv")

# Ensure ZIPs are strings and zero-padded
forecast_df["ZipCode"] = forecast_df["ZipCode"].astype(str).str[:5].str.zfill(5)

# Pivot forecast data so each ZIP has columns for 3/6/9/12-month forecasts
baseline = (
    forecast_df.pivot(index="ZipCode", columns="Forecast_Month", values="Forecast_Rent")
    .rename(columns={3: "Rent_3", 6: "Rent_6", 9: "Rent_9", 12: "Rent_12"})
    .reset_index()
)
print("✅ Forecast data ready:", baseline.shape)

# =========================================================
# 🧮 3️⃣ LOAD CURRENT RENT DATA
# =========================================================
current_df = pd.read_csv("Rental_zip_cleaned_long.csv", parse_dates=["Date"])
current_df["ZipCode"] = current_df["ZipCode"].astype(str).str[:5].str.zfill(5)

# Keep the latest record per ZIP
current_df = current_df.sort_values(["ZipCode", "Date"]).groupby("ZipCode").tail(1)
current_df = current_df[["ZipCode", "Rent", "City", "State", "Metro"]]
print("✅ Current rent data ready:", current_df.shape)

# =========================================================
# 🔗 4️⃣ MERGE CURRENT + FORECAST DATA
# =========================================================
merged = current_df.merge(baseline, on="ZipCode", how="left")
print("✅ Data merged:", merged.shape)

# =========================================================
# 🌎 5️⃣ LOAD ZIP CODE GEOMETRIES (LOCAL SHAPEFILE)
# =========================================================
shp_path = r"C:\Users\kbarc\OneDrive\Documents\Zip\cb_2018_us_zcta510_500k.shp"

if not os.path.exists(shp_path):
    raise FileNotFoundError(f"❌ Shapefile not found at {shp_path}. Please verify the path.")

shp = gpd.read_file(shp_path)[["ZCTA5CE10", "geometry"]]
shp = shp.rename(columns={"ZCTA5CE10": "ZipCode"})
shp["ZipCode"] = shp["ZipCode"].astype(str).str[:5].str.zfill(5)

print("✅ Loaded shapefile successfully!")
print("🔹 Shape count:", len(shp))

# Optional: simplify geometry for faster map rendering
shp["geometry"] = shp["geometry"].simplify(tolerance=0.01, preserve_topology=True)
print("✨ Geometry simplified for faster rendering")

# =========================================================
# 🧩 6️⃣ MERGE SHAPE + RENT DATA
# =========================================================
geo = shp.merge(merged, on="ZipCode", how="inner")
print("✅ Geo data ready:", geo.shape)

# =========================================================
# 🗺️ 7️⃣ INTERACTIVE RENT FORECAST MAP
# =========================================================
periods = {
    "Current Rent": "Rent",
    "3-Month Forecast": "Rent_3",
    "6-Month Forecast": "Rent_6",
    "9-Month Forecast": "Rent_9",
    "12-Month Forecast": "Rent_12"
}

# Default view
default_label = "Current Rent"
default_col = periods[default_label]

# Create the map
fig = px.choropleth_mapbox(
    geo,
    geojson=geo.__geo_interface__,
    locations="ZipCode",
    featureidkey="properties.ZipCode",
    color=default_col,
    color_continuous_scale="YlOrRd",
    range_color=(geo[default_col].quantile(0.05), geo[default_col].quantile(0.95)),
    hover_name="ZipCode",
    hover_data={
        "City": True,
        "State": True,
        "Metro": True,
        "Rent": ":,.0f",
        "Rent_3": ":,.0f",
        "Rent_6": ":,.0f",
        "Rent_9": ":,.0f",
        "Rent_12": ":,.0f",
    },
    mapbox_style="carto-positron",
    center={"lat": 37.8, "lon": -96},
    zoom=3.5,
    opacity=0.8,
    title=f"Rental Heat Map — {default_label}",
)

# =========================================================
# 🔘 8️⃣ FIXED DROPDOWN MENU (no JS errors)
# =========================================================
buttons = []
for label, col in periods.items():
    buttons.append(
        dict(
            label=label,
            method="update",
            args=[
                {"marker": [{"color": geo[col]}]},
                {"title": f"Rental Heat Map — {label}"}
            ],
        )
    )

fig.update_layout(
    updatemenus=[
        {
            "buttons": buttons,
            "direction": "down",
            "showactive": True,
            "x": 0.4,
            "xanchor": "left",
            "y": 1.05,
            "yanchor": "top",
        }
    ],
    margin={"r": 0, "t": 50, "l": 0, "b": 0},
)

# =========================================================
# 💾 9️⃣ DISPLAY & SAVE
# =========================================================
fig.show()
fig.write_html("forecast_rent_heatmap.html")
print("✅ Map saved as forecast_rent_heatmap.html")
